In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
# print(m/(m+n),'baseline accuracy')
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
file_directory = './data/parsed_data/'
save_directory = './data/lagged_data/'

In [ ]:
def get_feature_sequence(df,K=5,standarization=False):
    df  = df.sort_values('time').reset_index(drop=True)
    df['feature_final'] = df['feature_final'].apply(lambda a:np.float64(a.reshape(-1)[2:].reshape(1,-1)))
    if standarization:
        X = np.concatenate(list(df['feature_final']))
        X = preprocessing.StandardScaler().fit_transform(X)
        df['feature_final'] = list([np.array(a).reshape(1,-1) for a in X])
    time_of_day = df['Time of Day'].values[0]
    age = df['Gender'].values[0]
    user_id = df.user.values[0]
    all_data = []
    if df.shape[0]<=K:
        return pd.DataFrame([],columns=['features','user','label','day','time','Time of Day','Gender','start_time','end_time'])
    for i in range(K-1,df.shape[0],1):
        row = df.iloc[i]
        features = np.concatenate(list(df['feature_final'].iloc[i-K+1:i+1])).reshape(1,K,-1)
        label = df['Label'].iloc[i]
        day = df['day'].iloc[i]
        time = df['time'].iloc[i]
        start_time = df['start_time'].iloc[i-K+1]
        end_time = df['end_time'].iloc[i]
        all_data.append([features,user_id,label,day,time,time_of_day,age,start_time,end_time])
    return pd.DataFrame(all_data,columns=['features','user','label','day','time','Time of Day','Gender','start_time','end_time'])
# Ks = [30,60,75,90,120,150,180]
Ks = [5,10,15,20,25,30]
standarization = True
for f in os.listdir(file_directory):
    print(f)
    if f=='mRisk' or f[:3]=='lag' or f[0]=='.' or f[-6:]!='_new.p':
        continue
    obs = f.split('_')[4]
    prediction = f.split('_')[-2].split('.')[0]
    if int(prediction) not in [60]:
        continue
    if int(obs) not in [30]:
        continue
#     if os.path.isdir(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)):
#         continue
    if not os.path.isdir(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)):
        os.makedirs(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction))
    print(f,int(obs),int(prediction))
    data = pickle.load(open(file_directory+f,'rb'))
    for K1 in Ks:
#         K = (K1-int(obs))//2 + 1
#         if K<=1:
#             continue
        K = K1
        df_seq = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_feature_sequence)(df,K=K,standarization=standarization) for i,df in data.groupby('user',as_index=False)))
        saving_directory = save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)+'/'
        if not standarization:
            pickle.dump(df_seq,open(saving_directory+'lagged_'+str(K1)+'_windows_new.p','wb'))
        else:
            pickle.dump(df_seq,open(saving_directory+'lagged_'+str(K1)+'_windows_standardized_new.p','wb'))
        print(df_seq.shape,K1)
    print('done')
    print()

In [2]:
feature_names = ['Time of Day','Gender', 'Age', 'current_window_stress_p90', 'current_window_stress_p95', 'current_window_stress_p80',
 'current_window_stress_p5', 'current_window_stress_p10', 'current_window_stress_p20', 'current_window_stress_range_90_to_80',
 'current_window_stress_range_20_to_10', 'current_window_stress_range_90_to_10', 'current_window_stress_range_80_to_20', 'current_window_stress_median',
 'current_window_stress_iqr', 'current_window_stress_skew', 'current_window_stress_diff_p90', 'current_window_stress_diff_p95',
 'current_window_stress_diff_p80', 'current_window_stress_diff_p5', 'current_window_stress_diff_p10', 'current_window_stress_diff_p20',
 'current_window_stress_diff_range_90_to_80', 'current_window_stress_diff_range_20_to_10', 'current_window_stress_diff_range_90_to_10', 'current_window_stress_diff_range_80_to_20',
 'current_window_stress_diff_median', 'current_window_stress_diff_iqr', 'current_window_stress_diff_skew', 'time_since_last_visit_smoking_spot',
 'duration_of_stay_in_smoking_spot', 'fraction_of_minutes_spent_in_smoking_spots_out_of_observed', 'daily_stress_p90', 'daily_stress_p95',
 'daily_stress_p80', 'daily_stress_p5', 'daily_stress_p10', 'daily_stress_p20', 'daily_stress_range_90_to_80', 'daily_stress_range_20_to_10',
 'daily_stress_range_90_to_10', 'daily_stress_range_80_to_20', 'daily_stress_median', 'daily_stress_iqr', 'daily_stress_skew', 'percentage_of_stress', 'maximum_duration_of_current_stress_episode',
 'average_duration_of_current_stress_episode', 'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
 'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode', 'density_of_last_stress_episode',
 'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode', 'percentage_of_stress_before', 'percentage_of_active_',
 'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode', 'no_activity_till_now', 'time_since_last_activity',
 'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode', 'percentage_of_active_before', 'is_smoking', 'spread', 'distance_to_nearest_spot', 'time_spent_in_transition',
 'time_spent_in_smoking_spot']

from sklearn import preprocessing
use_standardization = True
use_current_stress = True
n_lag = 10
if use_standardization:
    data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows.p','rb'))
else:
    data  = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_'+str(n_lag)+'_windows'+'_standardized_new'+'.p','rb'))
n_t = data.iloc[0]['features'].shape[1]
n_f = data.iloc[0]['features'].shape[2]
y_time = data['time'].values
X = np.concatenate(list(data['features']))
n = X.shape[0]
X1 = X
y = data['label'].values
y = np.int64(np.array(y))
y[y>0] = 1
y[y<1] = -1
groups = data['user'].values
X_time = data['Time of Day'].values.reshape(-1,1)
X_gender = data.Gender.values.reshape(-1,1)
time_oh = preprocessing.OneHotEncoder().fit(X_time)
gender_oh = preprocessing.OneHotEncoder().fit(X_gender)
X_time = time_oh.transform(X_time).todense()
X_gender = gender_oh.transform(X_gender).todense()
if use_standardization:
    X1_all,y_all,groups_all,y_time_all,X_gender_all,X_time_all = [],[],[],[],[],[]
    for g in np.unique(groups):
        X1_all.append(X1[groups==g])
        y_all.extend(list(y[groups==g]))
        groups_all.extend(list(groups[groups==g]))
        y_time_all.extend(list(y_time[groups==g]))
        X_gender_all.append(preprocessing.StandardScaler().fit_transform(X_gender[groups==g]))
        X_time_all.append(preprocessing.StandardScaler().fit_transform(X_time[groups==g]))
    X1,y,groups,y_time,X_gender,X_time =  np.concatenate(X1_all),np.array(y_all),np.array(groups_all), \
    np.array(y_time_all),np.concatenate(X_gender_all),np.concatenate(X_time_all)

X_time = np.concatenate([np.expand_dims(X_time,axis=1)]*n_t,axis=1)
X_gender = np.concatenate([np.expand_dims(X_gender,axis=1)]*n_t,axis=1)
feature_names_temp = feature_names[2:]
if not use_current_stress:
    index_of_features_kept = np.array([max(k-2,0) for k,name in enumerate(feature_names_temp) if name.find('current_window_stress')==-1])
    X1 = X[:,:,index_of_features_kept]
# X = np.concatenate([X_time,X_gender,X1.reshape(X1.shape[0],-1)],axis=1)
# X.shape,y.shape,groups.shape,y_time.shape

In [3]:
time_feature_columns = [feature_names[0]+'=='+a for a in time_oh.categories_[0]]
gender_feature_columns = [feature_names[1]+'=='+a for a in gender_oh.categories_[0]]

feature_names = time_feature_columns+gender_feature_columns+feature_names_temp

X = np.concatenate([X1,X_time,X_gender],axis=2)

In [10]:
X.shape,y.shape,groups.shape,len(feature_names),Counter(y)

((68931, 10, 86), (68931,), (68931,), 86, Counter({-1: 67192, 1: 1739}))